# 策略概述

如果近期发生股份上破年线，且出现过日线放量，则可能有行情。后续操作需要人工判断

In [5]:
from alpha.notebook import *
from alpha.core.rsi_stats import rsiday, rsi30
import datetime
import os
from omicron.core.talib import cross

await init_notebook()

In [29]:
def cross_year(code, name, bars, results, frame_type):
    if len(bars) < 260:
        return
    
    close = bars["close"]
    ma = moving_average(close, 250)
    
    n = min(len(ma), 20)
    
    # 判断是否存在收盘价上穿年线
    pos_below = np.argwhere(close[-n:] < ma[-n:]).flatten()
    
    if len(pos_below) == 0:
        return
    
    pos_above = np.argwhere(close[-n:] > ma[-n:]).flatten()
    if len(pos_above) == 0:
        return
    
    if pos_below[0] > pos_above[-1]:
        return
    
    # 判断是否均线多头
    ma20 = moving_average(close, 20)[-1]
    ma30 = moving_average(close, 30)[-1]
    ma60 = moving_average(close, 60)[-1]
    
    if not (ma20 >= ma30 and ma30 >= ma60 and ma60 >= ma[-1]):
        return
    
    bias = close[-1] / ma[-1] - 1
    results.append((code, name, bias))

In [30]:
results = await scan(cross_year, 269, nstocks=-1, silent=False)

2022-01-06
progress: 500/4033, results: 15, elapsed: 7, ETA: 49
progress: 1000/4033, results: 33, elapsed: 10, ETA: 30
progress: 1500/4033, results: 48, elapsed: 27, ETA: 45
progress: 2000/4033, results: 59, elapsed: 29, ETA: 29
progress: 2500/4033, results: 63, elapsed: 65, ETA: 39
progress: 3000/4033, results: 79, elapsed: 68, ETA: 23
progress: 3500/4033, results: 91, elapsed: 72, ETA: 10
progress: 4000/4033, results: 101, elapsed: 78, ETA: 0


In [31]:
len(results)

102

In [23]:
def xp(bars, n, m):
    """
    n日内是否存在洗盘动作。
    
    如果在n日发生大于5%的下跌，此后m日碎步小阳（阳多于阴），则认为是洗盘
    """
    assert m < n - 1
    assert len(bars) > n + 1
    
    close = bars["close"][-n-1:]
 
    roc = close[1:]/close[:-1] - 1
    pos = np.argwhere(roc <= -0.05).flatten()
    if len(pos) == 0:
        return False
    
    pos = pos[-1]
    
    if pos + m > n:
        return False
    
    ups = np.count_nonzero(roc[pos + 1:] > 0)
    if ups/(n-pos) >= 0.6 or ups/(n-pos) >= 0.5 and roc[-1]>0:
        return True

In [34]:
df = pd.DataFrame(results, columns=["code", "name", "bias"])
df

,code,name,bias
0,000008.XSHE,神州高铁,0.226723
1,000029.XSHE,深深房A,0.124809
2,000419.XSHE,通程控股,0.101075
3,000529.XSHE,广弘控股,0.111728
4,000530.XSHE,冰山冷热,0.932389
...,...,...,...
97,603888.XSHG,新华网,0.270454
98,603918.XSHG,金桥信息,0.213076
99,603956.XSHG,威派格,0.068453
100,605128.XSHG,上海沿浦,0.248444


In [37]:
pd.options.display.max_rows = 500
df[(df.bias>0) & (df.bias<0.1)].sort_values(by='bias')

,code,name,bias
90,603128.XSHG,华贸物流,0.001220
58,300628.XSHE,亿联网络,0.009696
30,002471.XSHE,中超控股,0.017561
96,603856.XSHG,东宏股份,0.033797
93,603601.XSHG,再升科技,0.034086
12,000783.XSHE,长江证券,0.036169
33,002608.XSHE,江苏国信,0.039683
20,002216.XSHE,三全食品,0.046874
72,600420.XSHG,国药现代,0.049398
61,300778.XSHE,新城市,0.051665
